# Predictions

In [2]:
import torchvision
from PIL import Image, ImageDraw
import torch
from torchvision import transforms
from facenet_pytorch import MTCNN, extract_face

import numpy as np
import os
import matplotlib.pyplot as plt
import IPython.display

import torchvision.transforms.functional as F

In [3]:
# seleccionar GPU si està disponible..
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Running on {device}")

Running on cuda


### Model initialization and configuration

In [4]:
# Load the model and set to evaluation mode
PATH = 'model_UOC.pth'
new_model = torch.load(PATH)

new_model.eval()
new_model.to(device)

enum_classes= ['DanielValcarce', 'EnriqueGraziano', 'FranciscoMartin', 'GenisHeredia', 'JordiCasals', 'MarcelSerra', 'OriolJulia', 'PereClaver', 'RicardPeralta', 'RogerRoca']

In [5]:
# Create the face detector
mtcnn = MTCNN(
        image_size=224, margin=0, min_face_size=20, 
        thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True, 
        device=device, keep_all=True)

In [6]:
# Configure the transformations to be applied to images
transform = transforms.Compose([          
            transforms.Resize(224),   #mida d'entrada de la xarxa neuronal
            transforms.CenterCrop(224),    #important. Sinó dóna error perquè les imatges no son quadrades       
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])  #mitja i desviacio estandard

### Auxiliary functions

In [7]:
# Detect faces on an image
#
# detector: instance of MTCNN detector
# image: original image
def detect_faces(detector, image):
    boxes, probs, points = mtcnn.detect(image, landmarks=True)
    
    return boxes, probs, points

In [8]:
# Predict the classes from a face image
#
# model: instance of the model
# image: face image, cropped from the original image
# transformations: trasformations to be applied on image by the model

def predict(model, image, transformations):
    
    # if image is already a Tensor, convert it to a PIL Image in order to apply the model transformations    
    if isinstance(image, torch.FloatTensor):
        #print("already tensor. transorm to PIL")
        image=image.permute(1, 2, 0).numpy().astype('uint8')
        imagePIL= transforms.ToPILImage()(image)
    
          
    # Apply the transform to the image
    img = transformations(imagePIL) 
    img = img.to(device)

    # Pass the image to the model
    pred = model(img[None, ...])
    _, preds = torch.max(pred, 1)

    # Get probs
    sm = torch.nn.Softmax(dim=1)
    probs = sm(pred)

    resultat= enum_classes[preds]
    #print(resultat)
    return pred, preds, probs

In [9]:
# Display the original image with the face regions, classes and probabilities
#
# image: Original image
# detections: Array with all bboxes, class predicted and probilities
def show_image(image, detections):
    img_draw = image.copy()
    draw = ImageDraw.Draw(img_draw)
    
    for i, detection in enumerate(detections):
        box = detection[0] # Array with bbox values [x, y, w, h]
        draw.rectangle(box.tolist(), width=2)
        if detection[2]>0.98:
            caption = f"{detection[1]} - {detection[2]*100:.2f}%"
            draw.text((box[0], box[1]-10), caption)
        else:
            caption = "desconegut"
            draw.text((box[0], box[1]-10), caption)
        
    IPython.display.display(img_draw)

In [10]:
# Show a cropped face
#
# images: Array with cropped faces
def show_face_detected(images):
    fig = plt.figure(figsize=(15, 15))
    for i, image in enumerate(images):
        sub = fig.add_subplot(4, 4, i + 1)
        sub.imshow(images[i].permute(1, 2, 0).numpy().astype('uint8'))
        
#show_face_detected()

### Load image, preprocessing and predict

In [26]:
# Load image
image_path = os.path.join('data', 'original', 'img_3.jpg')
im = Image.open(image_path)

# Detect faces on images and get the regions for each face
face_regions, _, _ = detect_faces(mtcnn, im)

In [27]:
# Extract all faces detected in the original image
face_crops = []
for face in face_regions:
    crop = extract_face(im, face, image_size=224)
    face_crops.append(crop)

In [ ]:
show_face_detected(face_crops)

In [29]:
# Predict image
detections=[]

# face_crops: Array with only image faces
for i, crop in enumerate(face_crops):
    
#     fig = plt.figure(figsize=(10, 10))    
#     sub = fig.add_subplot(2, 1, 1)
#     sub.imshow(crop.permute(1, 2, 0).numpy().astype('uint8'))
    
    # Predict image with the cropped image
    _, preds, probs = predict(new_model, crop, transform)

    # Create the object with the region, class name and probability
    detection = [face_regions[i], enum_classes[preds], probs[0][preds].item()]
    detections.append(detection)

### Show image with predictions

In [ ]:
# Show image with the detections
show_image(im, detections)

# References

- [MTCNN](https://kpzhang93.github.io/MTCNN_face_detection_alignment/index.html) face landmark detector
- [Facenet](https://github.com/timesler/facenet-pytorch) PyTorch implementation